In [2]:
%load_ext autoreload
%autoreload 2
import torch, einops
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

from trl import SFTTrainer

# Prepare model

## Quantized  Low Ranking Adapters

In [2]:
model = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b-instruct", 
        quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        ), 
        device_map={"": 0}, 
        trust_remote_code=True
    )
model.config.use_cache = False

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32, #rank of the matrix
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ],
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.99s/it]


In [3]:
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

Found cached dataset json (/home/isaid/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [4]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.truncate_sequences = True
tokenizer.pad_token = tokenizer.eos_token
max_length = tokenizer.model_max_length
max_length

2048

## Training args

In [5]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    
    fp16=True,
    
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=300,
    warmup_ratio=0.03,
    # lr_scheduler_type="constant",
    lr_scheduler_type="constant",
    
    logging_steps=10,
    
    save_steps=10,
    output_dir="./training_results",
    report_to="tensorboard",
)

In [8]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

/home/isaid/arkham/failsafe/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [9]:
trainer.train()

Step,Training Loss
10,1.752300
20,1.593100
30,1.443000
40,1.440700
50,1.418100
60,1.431200
70,1.426200
80,1.414400
90,1.403500
100,1.364300


/home/isaid/arkham/failsafe/lib/python3.10/site-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


TrainOutput(global_step=300, training_loss=1.4042439238230386, metrics={'train_runtime': 3829.4496, 'train_samples_per_second': 0.313, 'train_steps_per_second': 0.078, 'total_flos': 4.98187414536192e+16, 'train_loss': 1.4042439238230386, 'epoch': 0.12})

Notice, that run 240 had the lowest loss rate, hence we will take this as the fine tuned model.

In [4]:
%load_ext tensorboard
%tensorboard --logdir training_results

Reusing TensorBoard on port 6006 (pid 2771), started 1:30:01 ago. (Use '!kill 2771' to kill it.)